In [6]:
import glob
import math
import os
import random
import json
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from datetime import datetime as dt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, LearningRateScheduler
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from losses import categorical_focal_loss

from tensorflow.keras.applications.densenet import preprocess_input
import tensorflow as tf
print("TF Version:", tf.__version__)

TF Version: 1.15.0


In [8]:
train_dir = ''
test_dir = ''

model_json = 
model_weight = 

WIDTH = 300
HEIGHT = 400
BATCH = 4

SyntaxError: invalid syntax (<ipython-input-8-4bb3a87e34af>, line 4)

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_flow = train_datagen.flow_from_directory(
    train_dir,
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH,
    subset='training',
    shuffle=True)

val_flow = train_datagen.flow_from_directory(
    '/media/newhd/11_cat_train',
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH,
    subset='validation',
    shuffle=False)

NameError: name 'train_dir' is not defined

In [ ]:
file_list = glob.glob("")
img_path = random.choice(file_list)
img = load_img(img_path)
car_class = img_path.split("/")[-2]
plt.imshow(img)
plt.axis("off")
plt.title(car_class, fontsize=16)